In [ ]:
%run '/content/drive/MyDrive/Colab Notebooks/Hamoye Intership/classification/log_regression.ipynb'

In [16]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(log_reg, normalized_train, y_balanced, cv=5, scoring='f1_macro')
scores

array([0.45545128, 0.49872123, 0.44994499, 0.40241449, 0.51391162])

k-Fold Crosss Validation

In [22]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
kf = KFold(n_splits=5)
kf.split(normalized_train)
f1_scores = []

def _run_():
  for train_index, test_index in kf.split(normalized_train):
    x_train, x_test = [normalized_train.iloc[train_index], 
                        normalized_train.iloc[test_index]]
    y_train, y_test = [y_balanced.iloc[train_index], 
                        y_balanced.iloc[test_index]]
    model = LogisticRegression().fit(x_train, y_train)
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A')*100)
  return f1_scores

_run_()

[58.156028368794324,
 60.99290780141844,
 47.36842105263157,
 53.03030303030303,
 0.0]

Stratified K-Fold Cross Validation

In [37]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []

def _run_():
  for train_index, test_index in skf.split(normalized_train, y_balanced):
    x_train, x_test = np.array(normalized_train)[train_index], np.array(normalized_train)[test_index]
    y_train, y_test = np.array(y_balanced)[train_index], np.array(y_balanced)[test_index]
    model = LogisticRegression().fit(x_train, y_train)
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A'))
  return f1_scores

_run_()

[0.5098039215686274,
 0.5333333333333333,
 0.46464646464646464,
 0.5423728813559322,
 0.5950413223140496]

Leave One Out Cross Validation (LOOCV)

In [41]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
score = cross_val_score(LogisticRegression(), normalized_train, y_balanced, 
                        cv=loo, scoring='f1_macro')
average_score = score.mean() * 100
average_score

48.795180722891565

Confusion Matrix

In [117]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

new_pred = log_reg.predict(normalized_test)
cn_matrix = confusion_matrix(y_true=y_test, y_pred=new_pred, labels=['2A', '3A'])
names = ['True Neg','False Pos','False Neg','True Pos']

counts = [int('{0:0.0f}'.format(value)) for value in cn_matrix.flatten()]
labels = [f"{v1}\n{v2}" for v1, v2 in zip(names, counts)]
labels = np.asarray(names).reshape(2,2)
sns.heatmap(cn_matrix, annot=np.array(names).reshape(2,2), linewidths=2, annot_kws={"size": 12})
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()


array([[31, 45],
       [45, 56]])

Accuracy

![\Large accuracy=\frac{TP\+TN}{TP\+TN\+FP\+FN}](https://latex.codecogs.com/svg.latex?\Large&space;accuracy=\frac{TP\+TN}{TP\+TN\+FP\+FN})

In [118]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_pred)
print("accuracy: {}".format(round(accuracy * 100), 2))

accuracy: 49


Precision

In [120]:
precision = precision_score(y_true=y_test, y_pred=new_pred, pos_label='2A')
print("precision: {}".format(round(precision * 100), 2))

precision: 41


Recall

In [ ]:
recall = recall_score(y_true=y_test, y_pred=new_pred)
print("recall: {}".format(round(recall * 100), 2))

There is a trade-off between Recall and Precision. It is impossible to maximise both metrics simultaneously because an increase in recall decreases precision